In [1]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 661.2/661.2 kB 8.9 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 443.6/443.6 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.5/314.5 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 15.2 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0


In [5]:
from openai import OpenAI

In [6]:
client = OpenAI(api_key=API_KEY)

In [7]:
from pathlib import Path

JSONPATH = Path("/Users/joregan/Playing/hsi/annotations/final_resolved")

In [26]:
SKIPPABLES = ["room"]

def process_item(item):
    utt_key = "utterance_type"
    if not utt_key in item["high_level"]:
        utt_key = "topic_name"
    if item["high_level"][utt_key] == "conversation_generic":
        return {}
    current = item["high_level"]["current_topic"]
    if current in SKIPPABLES:
        return {}
    snippet = item["snippet"]
    return {
        "topic:": current,
        "snippet": snippet
    }

In [29]:
import json

alldata = []

for jsonfile in JSONPATH.glob("*.json"):
    stem = jsonfile.stem
    parts = stem.split("_")
    person = parts[1]
    room = parts[3]

    with open(jsonfile) as f:
        data = json.load(f)

    output = {
        "person": person,
        "room": room,
        "items": []
    }
    for item in data.keys():
        processed = process_item(data[item])
        if processed and processed != {}:
            output["items"].append(processed)
    alldata.append(output)

In [31]:
with open("/tmp/hsi.json", "w") as f:
    json.dump(alldata, f, indent=4)

In [32]:
test = alldata[0]

In [ ]:
PROMPT = f"""
Below is some data from a conversation. The data is in JSON format.
The data contains a list of items, each with a topic and a snippet. The topic is the subject of the conversation, and the snippet is a short excerpt from the conversation.
Continue the conversation by writing a new snippet that is relevant to the topic. The new snippet should contain a reference to the topic and should be a natural continuation of the conversation.
The new snippet should be in the same format as the original snippets, and should be a short paragraph of text, of between 10 and 50 words.
There should only be one reference to the topic in the new snippet. It should be a natural part of the conversation, and should not be forced or awkward. It should be marked in boldface using markdown syntax.
The reference to the topic should be suitable for instructing a person to point at it in a conversation. It can be a noun phrase, or a pronoun, but it should be clear and unambiguous. If the reference is a noun phrase, it should include a determiner, such as "this" or "that". If the reference is a pronoun, it should be clear what it refers to.
If another topic is mentioned in the new snippet and the spatial relationship is very clear, it is ok to reuse that spatial reference.
```json
{test}
```
"""

In [35]:
response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {
            "role": "user",
            "content": [
                { "type": "text", "text": PROMPT },
            ]
        }
    ]
)

In [36]:
response

ChatCompletion(id='chatcmpl-BRGKBf83FizBn2cuY40CfosbbuwGm', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='```json\n{\'topic:\': \'Book_28\', \'snippet\': "Yeah, I noticed **that novel** when I came in, and it seemed interesting. It must belong to someone who really enjoys detective stories, don\'t you think?"}\n```', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1745837307, model='gpt-4o-2024-08-06', object='chat.completion', service_tier='default', system_fingerprint='fp_90122d973c', usage=CompletionUsage(completion_tokens=52, prompt_tokens=1029, total_tokens=1081, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))

In [43]:
response_text = response.choices[0].message.content

In [44]:
response_text

'```json\n{\'topic:\': \'Book_28\', \'snippet\': "Yeah, I noticed **that novel** when I came in, and it seemed interesting. It must belong to someone who really enjoys detective stories, don\'t you think?"}\n```'

In [47]:
def clean_response(response_text):
    response_text = response_text.replace("```json", "")
    response_text = response_text.replace("```", "")
    response_text = response_text.strip()
    response_text = response_text.replace("'", '"')
    try:
        return json.loads(response_text)
    except json.JSONDecodeError:
        print(response_text)

In [48]:
clean_response(response_text)

{"topic:": "Book_28", "snippet": "Yeah, I noticed **that novel** when I came in, and it seemed interesting. It must belong to someone who really enjoys detective stories, don"t you think?"}
